In [2]:
cd ..

c:\SMIT3.0


In [3]:
from itertools import count
from deep_translator import GoogleTranslator
from difflib import get_close_matches
import json
import re
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import pyodbc
import csv

from pythainlp import sent_tokenize, word_tokenize, correct, spell, Tokenizer
import gensim

from gensim.models import Word2Vec

from sklearn.neighbors import NearestCentroid
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt

import numpy as np
from pythainlp.util import dict_trie, Trie 
from matplotlib.colors import ListedColormap

Get Data from Database

In [ ]:
Custom_Dict = pd.read_csv('./SMIT_Data/DataForModel/Raw_Dictionary.csv', encoding='utf-8')
DictCorrect = Custom_Dict['correct'].tolist()

DictCorrect = list(set(DictCorrect))

modelPath = "./Model/SentenceTransformer"
model = SentenceTransformer(modelPath)

trie = Trie(DictCorrect)

# connect_db = pyodbc.connect(Driver = "ODBC Driver 17 for SQL Server",
#             Server = "smitazure.database.windows.net",
#             Database = "SafetyAudit",
#             uid = 'smitadmin',
#             pwd = 'Abc12345',
#             Trusted_Connection = 'no') 

connect_db = pyodbc.connect(Driver = "ODBC Driver 17 for SQL Server",
                            Server = "TONY",
                            Database = "SMIT3",
                            uid = 'Local_SMIT3.0',
                            pwd = 'Tony123456',
                            Trusted_Connection = 'yes')  

cursor = connect_db.cursor()

GetOldID = pd.read_sql('''
                        SELECT MostSimTOF(CAST([ID] AS int))
                        FROM [dbo].[LOG_Finding]
                        WHERE ID > 113 AND ID != 131 AND ID != 5057 AND ID != 5058 AND ID != 190 AND ID != 1483 AND ID != 1486 AND ID != 1974 AND ID != 132 AND 
	                        (AuditResult = 'Need to Improve' or AuditResult = 'Non-conform') 
	                        AND Corrective != '';''', connect_db)

GetOldID = GetOldID[''].tolist()[0]

# Query = '''SELECT [Area], [AuditResult], [Finding], [Corrective] FROM [LOG_Finding] 
#             WHERE ID > 113 AND ID != 131 AND ID != 5057 AND ID != 5058 AND ID != 190 AND ID != 1483 AND 
#                 ID != 1486 AND ID != 1974 AND ID != 132 AND 
#                 (AuditResult = 'Need to Improve' or AuditResult = 'Non-conform') AND 
#                 Corrective != '' AND 
#                 ID > '''


# tbFinding = pd.read_sql(Query+str(GetOldID), connect_db)

Query = '''SELECT [Area], [AuditResult], [Finding], [Corrective] FROM [LOG_Finding] 
            WHERE ID > 113 AND ID != 131 AND ID != 5057 AND ID != 5058 AND ID != 190 AND ID != 1483 AND 
                ID != 1486 AND ID != 1974 AND ID != 132 AND 
                (AuditResult = 'Need to Improve' or AuditResult = 'Non-conform') AND 
                Corrective != '' '''

tbFinding = pd.read_sql(Query, connect_db)


SafetyAudit = pd.read_csv("./SMIT_Data/AllRawSafetyAudit_New.csv", encoding='utf-8')

# # tbFinding = pd.read_csv('./SMIT_Data/Classification_tbFinding/tbFinding.csv', encoding='utf-8')[TotalOldClassification_Finding:]

Finding = tbFinding['Finding'].tolist()
Translate_tbFinding = []

for sentence in Finding:
    Translate_tbFinding.append(GoogleTranslator(source = 'auto', target = 'en').translate(sentence))

Get Data from CSV

In [4]:
tbFinding = pd.read_csv("./SMIT_Data/Classification_tbFinding/tbFinding.csv", encoding="utf-8")
tbPermit = pd.read_csv("./SMIT_Data/Classification_tbFinding/tbPermit.csv", encoding="utf-8")

tbFinding_NonConform = tbFinding[tbFinding["AuditResult"] == "Non-conform"]
tbFinding_NeedToImprove = tbFinding[tbFinding["AuditResult"] == "Need to Improve"]

tbFinding = [tbFinding_NonConform, tbFinding_NeedToImprove]
tbFinding = pd.concat(tbFinding)

UnwantedID = [131, 5057, 5058, 90, 1483, 1486, 1974, 132]
tbFinding = tbFinding[tbFinding['ID'] > 113]

for i in UnwantedID:
    tbFinding = tbFinding[tbFinding['ID'] != i]

tbFindingTitle = tbFinding['Title'].tolist()
tbFindingArea = tbFinding['Area'].tolist()
tbFindingSubArea = []
tbFindingContractor = []
tbFindingTof = []
tbFindingTopic = []
tbFindingDetail = []
tbFindingAuditResult = tbFinding["AuditResult"].tolist()
tbFindingFinding = tbFinding["Finding"].tolist()


tbPermitTitle = tbPermit['Title'].tolist()
tbPermitDetail = tbPermit['Detail'].tolist()

for i in tbFindingTitle:
    TemptbFindingDetail = tbPermit[tbPermit['Title'] == i]
    tbFindingDetail.append(TemptbFindingDetail['Detail'].tolist())

print(len(tbFinding_NonConform), len(tbFinding_NeedToImprove), len(tbFinding), len(tbFindingDetail))

Custom_Dict = pd.read_csv('./SMIT_Data/DataForModel/Raw_Dictionary.csv', encoding='utf-8')
DictCorrect = Custom_Dict['correct'].tolist()

DictCorrect = list(set(DictCorrect))
SafetyAudit = pd.read_csv("./SMIT_Data/AllRawSafetyAudit_New.csv", encoding='utf-8')

Translate_tbFinding = []

for sentence in tbFindingFinding:
    # try:
    Translate_tbFinding.append(GoogleTranslator(source = 'auto', target = 'en').translate(sentence))
    # except:


753 377 1100 1100


NotValidPayload: 12 --> text must be a valid text with maximum 5000 character, otherwise it cannot be translated

Create Dict for 5 Clusters : TypeOfFinding

In [ ]:
# ------------------ Prepared Cluster : Unsafe Action ------------------

ENData_UnsafeAction = (SafetyAudit.loc[SafetyAudit["TypeOfFinding"] == "Unsafe Action", "TransCleansingDetails"]).tolist()
THData_UnsafeAction = (SafetyAudit.loc[SafetyAudit["TypeOfFinding"] == "Unsafe Action", "CleansingDetails"]).tolist()
Data_UnsafeAction = ' '.join(ENData_UnsafeAction+THData_UnsafeAction)

CountVector = CountVectorizer(stop_words='english') 
doc_array = CountVector.fit_transform([Data_UnsafeAction]).toarray() 

# ListWords_UnsafeAction = list(set(list(CountVector.get_feature_names_out()))) 

ListWords_UnsafeAction = list(set(word_tokenize(Data_UnsafeAction, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Unsafe Condition ------------------

ENData_UnsafeCondition = (SafetyAudit.loc[SafetyAudit["TypeOfFinding"] == "Unsafe Condition", "TransCleansingDetails"]).tolist()
THData_UnsafeCondition = (SafetyAudit.loc[SafetyAudit["TypeOfFinding"] == "Unsafe Condition", "CleansingDetails"]).tolist()
Data_UnsafeCondition = ' '.join(ENData_UnsafeCondition+THData_UnsafeCondition)

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_UnsafeCondition]).toarray() 

# ListWords_UnsafeCondition = list(set(list(CountVector.get_feature_names_out()))) 

ListWords_UnsafeCondition = list(set(word_tokenize(Data_UnsafeCondition, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Near Miss ------------------

ENData_NearMiss = (SafetyAudit.loc[SafetyAudit["TypeOfFinding"] == "Near Miss", "TransCleansingDetails"]).tolist()
THData_NearMiss = (SafetyAudit.loc[SafetyAudit["TypeOfFinding"] == "Near Miss", "CleansingDetails"]).tolist()
Data_NearMiss = ' '.join(ENData_NearMiss+THData_NearMiss)

CountVector = CountVectorizer(stop_words='english', vocabulary=DictCorrect) 
doc_array = CountVector.fit_transform([Data_NearMiss]).toarray() 

# ListWords_NearMiss = list(set(list(CountVector.get_feature_names_out()))) 

ListWords_NearMiss = list(set(word_tokenize(Data_NearMiss, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : HNM ------------------

ENData_HNM = (SafetyAudit.loc[SafetyAudit["TypeOfFinding"] == "HNM", "TransCleansingDetails"]).tolist()
THData_HNM = (SafetyAudit.loc[SafetyAudit["TypeOfFinding"] == "HNM", "CleansingDetails"]).tolist()
Data_HNM = ' '.join(ENData_HNM+THData_HNM)

CountVector = CountVectorizer(stop_words='english', vocabulary=DictCorrect) 
doc_array = CountVector.fit_transform([Data_HNM]).toarray() 

# ListWords_HNM = list(set(list(CountVector.get_feature_names_out()))) 

ListWords_HNM = list(set(word_tokenize(Data_HNM, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Accident ------------------
ENData_Accident = (SafetyAudit.loc[SafetyAudit["TypeOfFinding"] == "Accident", "TransCleansingDetails"]).tolist() 
THData_Accident = (SafetyAudit.loc[SafetyAudit["TypeOfFinding"] == "Accident", "CleansingDetails"]).tolist() 
Data_Accident = ' '.join(ENData_Accident+THData_Accident) 

CountVector = CountVectorizer(stop_words='english', vocabulary=DictCorrect) 
doc_array = CountVector.fit_transform([Data_Accident]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out())))   

ListWords_Accident = list(set(word_tokenize(Data_Accident, custom_dict=trie, engine='newmm')))

Create Dict for 5 Clusters : Topic

In [ ]:
# SafetyAudit = pd.read_csv("./SMIT_Data/AllRawSafetyAudit.csv", encoding='utf-8')

# ------------------ Prepared Cluster : LOTO/ LB ------------------

ENData_LOTO = (SafetyAudit.loc[SafetyAudit["Topic"] == "LOTO/ LB", "TransCleansingDetails"]).tolist()
THData_LOTO = (SafetyAudit.loc[SafetyAudit["Topic"] == "LOTO/ LB", "CleansingDetails"]).tolist()
Data_LOTO = ' '.join(ENData_LOTO+THData_LOTO)

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_LOTO]).toarray() 

# ListWords_UnsafeAction = list(set(list(CountVector.get_feature_names_out()))) 

ListWords_LOTO = list(set(word_tokenize(Data_LOTO, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Work at height ------------------

ENData_WAH = (SafetyAudit.loc[SafetyAudit["Topic"] == "Work at height", "TransCleansingDetails"]).tolist()
THData_WAH = (SafetyAudit.loc[SafetyAudit["Topic"] == "Work at height", "CleansingDetails"]).tolist()
Data_WAH = ' '.join(ENData_WAH+THData_WAH)

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_WAH]).toarray() 

# ListWords_UnsafeCondition = list(set(list(CountVector.get_feature_names_out()))) 

ListWords_WAH = list(set(word_tokenize(Data_WAH, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Scaffolding ------------------

ENData_Scaffolding = (SafetyAudit.loc[SafetyAudit["Topic"] == "Scaffolding", "TransCleansingDetails"]).tolist()
THData_Scaffolding = (SafetyAudit.loc[SafetyAudit["Topic"] == "Scaffolding", "CleansingDetails"]).tolist()
Data_Scaffolding = ' '.join(ENData_Scaffolding+THData_Scaffolding)

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_Scaffolding]).toarray() 

# ListWords_NearMiss = list(set(list(CountVector.get_feature_names_out()))) 

ListWords_Scaffolding = list(set(word_tokenize(Data_Scaffolding, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Transportation ------------------

ENData_Transportation = (SafetyAudit.loc[SafetyAudit["Topic"] == "Transportation", "TransCleansingDetails"]).tolist()
THData_Transportation = (SafetyAudit.loc[SafetyAudit["Topic"] == "Transportation", "CleansingDetails"]).tolist()
Data_Transportation = ' '.join(ENData_Transportation+THData_Transportation)

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_Transportation]).toarray() 

# ListWords_HNM = list(set(list(CountVector.get_feature_names_out()))) 

ListWords_Transportation = list(set(word_tokenize(Data_Transportation, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : PTW & JSA ------------------
ENData_PTW_JSA = (SafetyAudit.loc[SafetyAudit["Topic"] == "PTW & JSA", "TransCleansingDetails"]).tolist() 
THData_PTW_JSA = (SafetyAudit.loc[SafetyAudit["Topic"] == "PTW & JSA", "CleansingDetails"]).tolist() 
Data_PTW_JSA = ' '.join(ENData_PTW_JSA+THData_PTW_JSA) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_PTW_JSA]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out())))   

ListWords_PTW_JSA = list(set(word_tokenize(Data_PTW_JSA, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Process & Operation ------------------
ENData_ProcessOperation = (SafetyAudit.loc[SafetyAudit["Topic"] == "Process & Operation", "TransCleansingDetails"]).tolist() 
THData_ProcessOperation = (SafetyAudit.loc[SafetyAudit["Topic"] == "Process & Operation", "CleansingDetails"]).tolist() 
Data_ProcessOperation = ' '.join(ENData_ProcessOperation+THData_ProcessOperation) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_ProcessOperation]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out())))   

ListWords_ProcessOperation = list(set(word_tokenize(Data_ProcessOperation, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Radiation ------------------
ENData_Radiation = (SafetyAudit.loc[SafetyAudit["Topic"] == "Radiation", "TransCleansingDetails"]).tolist() 
THData_Radiation = (SafetyAudit.loc[SafetyAudit["Topic"] == "Radiation", "CleansingDetails"]).tolist() 
Data_Radiation = ' '.join(ENData_Radiation+THData_Radiation) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_Radiation]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out())))   

ListWords_Radiation = list(set(word_tokenize(Data_Radiation, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Radiation ------------------
ENData_Others = (SafetyAudit.loc[SafetyAudit["Topic"] == "Others", "TransCleansingDetails"]).tolist() 
THData_Others = (SafetyAudit.loc[SafetyAudit["Topic"] == "Others", "CleansingDetails"]).tolist() 
Data_Others = ' '.join(ENData_Others+THData_Others) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_Others]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out())))   

ListWords_Others = list(set(word_tokenize(Data_Others, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Lifting ------------------
ENData_Lifting = (SafetyAudit.loc[SafetyAudit["Topic"] == "Lifting", "TransCleansingDetails"]).tolist() 
THData_Lifting = (SafetyAudit.loc[SafetyAudit["Topic"] == "Lifting", "CleansingDetails"]).tolist() 
Data_Lifting = ' '.join(ENData_Lifting+THData_Lifting) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_Lifting]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out())))   

ListWords_Lifting = list(set(word_tokenize(Data_Lifting, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Housekeeping ------------------
ENData_Housekeeping = (SafetyAudit.loc[SafetyAudit["Topic"] == "Housekeeping", "TransCleansingDetails"]).tolist() 
THData_Housekeeping = (SafetyAudit.loc[SafetyAudit["Topic"] == "Housekeeping", "CleansingDetails"]).tolist() 
Data_Housekeeping = ' '.join(ENData_Housekeeping+THData_Housekeeping) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_Housekeeping]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out())))   

ListWords_Housekeeping = list(set(word_tokenize(Data_Housekeeping, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Tools & Equipment ------------------
ENData_ToolsEquipment = (SafetyAudit.loc[SafetyAudit["Topic"] == "Tools & Equipment", "TransCleansingDetails"]).tolist() 
THData_ToolsEquipment = (SafetyAudit.loc[SafetyAudit["Topic"] == "Tools & Equipment", "CleansingDetails"]).tolist() 
Data_ToolsEquipment = ' '.join(ENData_ToolsEquipment+THData_ToolsEquipment) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_ToolsEquipment]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out())))   

ListWords_ToolsEquipment = list(set(word_tokenize(Data_ToolsEquipment, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Hot Work ------------------
ENData_HotWork = (SafetyAudit.loc[SafetyAudit["Topic"] == "Hot Work", "TransCleansingDetails"]).tolist() 
THData_HotWork = (SafetyAudit.loc[SafetyAudit["Topic"] == "Hot Work", "CleansingDetails"]).tolist() 
Data_HotWork = ' '.join(ENData_HotWork+THData_HotWork) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_HotWork]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out())))   

ListWords_HotWork = list(set(word_tokenize(Data_HotWork, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Excavation ------------------
ENData_Excavation = (SafetyAudit.loc[SafetyAudit["Topic"] == "Excavation", "TransCleansingDetails"]).tolist() 
THData_Excavation = (SafetyAudit.loc[SafetyAudit["Topic"] == "Excavation", "CleansingDetails"]).tolist() 
Data_Excavation = ' '.join(ENData_Excavation+THData_Excavation) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_Excavation]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out())))   

ListWords_Excavation = list(set(word_tokenize(Data_Excavation, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : CSE ------------------
ENData_CSE = (SafetyAudit.loc[SafetyAudit["Topic"] == "CSE", "TransCleansingDetails"]).tolist() 
THData_CSE = (SafetyAudit.loc[SafetyAudit["Topic"] == "CSE", "CleansingDetails"]).tolist() 
Data_CSE = ' '.join(ENData_CSE+THData_CSE) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_CSE]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out())))   

ListWords_CSE = list(set(word_tokenize(Data_CSE, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Electrical & Grounding ------------------
ENData_ElectricalGrounding = (SafetyAudit.loc[SafetyAudit["Topic"] == "Electrical & Grounding", "TransCleansingDetails"]).tolist() 
THData_ElectricalGrounding = (SafetyAudit.loc[SafetyAudit["Topic"] == "Electrical & Grounding", "CleansingDetails"]).tolist() 
Data_ElectricalGrounding = ' '.join(ENData_ElectricalGrounding+THData_ElectricalGrounding) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_ElectricalGrounding]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out())))   

ListWords_ElectricalGrounding = list(set(word_tokenize(Data_ElectricalGrounding, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Paint/ Coat/ Blast ------------------
ENData_PaintCoatBlast = (SafetyAudit.loc[SafetyAudit["Topic"] == "Paint/ Coat/ Blast", "TransCleansingDetails"]).tolist() 
THData_PaintCoatBlast = (SafetyAudit.loc[SafetyAudit["Topic"] == "Paint/ Coat/ Blast", "CleansingDetails"]).tolist() 
Data_PaintCoatBlast = ' '.join(ENData_PaintCoatBlast+THData_PaintCoatBlast) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_PaintCoatBlast]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out()))) 

ListWords_PaintCoatBlast = list(set(word_tokenize(Data_PaintCoatBlast, custom_dict=trie, engine='newmm'))) 

# ------------------ Prepared Cluster : Chemical Work ------------------
ENData_ChemicalWork = (SafetyAudit.loc[SafetyAudit["Topic"] == "Chemical Work", "TransCleansingDetails"]).tolist() 
THData_ChemicalWork = (SafetyAudit.loc[SafetyAudit["Topic"] == "Chemical Work", "CleansingDetails"]).tolist() 
Data_ChemicalWork = ' '.join(ENData_ChemicalWork+THData_ChemicalWork) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_ChemicalWork]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out()))) 

ListWords_ChemicalWork = list(set(word_tokenize(Data_ChemicalWork, custom_dict=trie, engine='newmm'))) 

# ------------------ Prepared Cluster : Safety Management ------------------
ENData_SafetyManagement = (SafetyAudit.loc[SafetyAudit["Topic"] == "Safety Management", "TransCleansingDetails"]).tolist() 
THData_SafetyManagement = (SafetyAudit.loc[SafetyAudit["Topic"] == "Safety Management", "CleansingDetails"]).tolist() 
Data_SafetyManagement = ' '.join(ENData_SafetyManagement+THData_SafetyManagement) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_SafetyManagement]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out()))) 

ListWords_SafetyManagement = list(set(word_tokenize(Data_SafetyManagement, custom_dict=trie, engine='newmm'))) 

# ------------------ Prepared Cluster : PPE ------------------
ENData_PPE = (SafetyAudit.loc[SafetyAudit["Topic"] == "PPE", "TransCleansingDetails"]).tolist() 
THData_PPE = (SafetyAudit.loc[SafetyAudit["Topic"] == "PPE", "CleansingDetails"]).tolist() 
Data_PPE = ' '.join(ENData_PPE+THData_PPE) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_PPE]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out()))) 

ListWords_PPE = list(set(word_tokenize(Data_PPE, custom_dict = trie, engine='newmm'))) 

# ------------------ Prepared Cluster : Water jet ------------------
ENData_WaterJet = (SafetyAudit.loc[SafetyAudit["Topic"] == "Water jet", "TransCleansingDetails"]).tolist() 
THData_WaterJet = (SafetyAudit.loc[SafetyAudit["Topic"] == "Water jet", "CleansingDetails"]).tolist() 
Data_WaterJet = ' '.join(ENData_WaterJet+THData_WaterJet) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_WaterJet]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out()))) 

ListWords_WaterJet = list(set(word_tokenize(Data_WaterJet, custom_dict=trie, engine='newmm'))) 

# ------------------ Prepared Cluster : Pressure test ------------------
ENData_PressureTest = (SafetyAudit.loc[SafetyAudit["Topic"] == "Pressure test", "TransCleansingDetails"]).tolist() 
THData_PressureTest = (SafetyAudit.loc[SafetyAudit["Topic"] == "Pressure test", "CleansingDetails"]).tolist() 
Data_PressureTest = ' '.join(ENData_PressureTest+THData_PressureTest) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_PressureTest]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out()))) 

ListWords_PressureTest = list(set(word_tokenize(Data_PressureTest, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : SL Performance ------------------
ENData_SLPerformance = (SafetyAudit.loc[SafetyAudit["Topic"] == "SL Performance", "TransCleansingDetails"]).tolist() 
THData_SLPerformance = (SafetyAudit.loc[SafetyAudit["Topic"] == "SL Performance", "CleansingDetails"]).tolist() 
Data_SLPerformance = ' '.join(ENData_SLPerformance+THData_SLPerformance) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_SLPerformance]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out())))   

ListWords_SLPerformance = list(set(word_tokenize(Data_SLPerformance, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Work Procedure ------------------
ENData_WorkProcedure = (SafetyAudit.loc[SafetyAudit["Topic"] == "Work Procedure", "TransCleansingDetails"]).tolist() 
THData_WorkProcedure = (SafetyAudit.loc[SafetyAudit["Topic"] == "Work Procedure", "CleansingDetails"]).tolist() 
Data_WorkProcedure = ' '.join(ENData_WorkProcedure+THData_WorkProcedure) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_WorkProcedure]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out()))) 

ListWords_WorkProcedure = list(set(word_tokenize(Data_WorkProcedure, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Civil ------------------
ENData_Civil = (SafetyAudit.loc[SafetyAudit["Topic"] == "Civil", "TransCleansingDetails"]).tolist() 
THData_Civil = (SafetyAudit.loc[SafetyAudit["Topic"] == "Civil", "CleansingDetails"]).tolist() 
Data_Civil = ' '.join(ENData_Civil+THData_Civil) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_Civil]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out())))   

ListWords_Civil = list(set(word_tokenize(Data_Civil, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Insulation ------------------
ENData_Insulation = (SafetyAudit.loc[SafetyAudit["Topic"] == "Insulation", "TransCleansingDetails"]).tolist() 
THData_Insulation = (SafetyAudit.loc[SafetyAudit["Topic"] == "Insulation", "CleansingDetails"]).tolist() 
Data_Insulation = ' '.join(ENData_Insulation+THData_Insulation) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_Insulation]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out())))   

ListWords_Insulation = list(set(word_tokenize(Data_Insulation, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Environmental ------------------
ENData_Environmental = (SafetyAudit.loc[SafetyAudit["Topic"] == "Environmental", "TransCleansingDetails"]).tolist() 
THData_Environmental = (SafetyAudit.loc[SafetyAudit["Topic"] == "Environmental", "CleansingDetails"]).tolist() 
Data_Environmental = ' '.join(ENData_Environmental+THData_Environmental) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_Environmental]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out())))   

ListWords_Environmental = list(set(word_tokenize(Data_Environmental, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Installation/ Alignment ------------------
ENData_InstallationAlignment = (SafetyAudit.loc[SafetyAudit["Topic"] == "Installation/ Alignment", "TransCleansingDetails"]).tolist() 
THData_InstallationAlignment = (SafetyAudit.loc[SafetyAudit["Topic"] == "Installation/ Alignment", "CleansingDetails"]).tolist() 
Data_InstallationAlignment = ' '.join(ENData_InstallationAlignment+THData_InstallationAlignment) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_InstallationAlignment]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out())))   

ListWords_InstallationAlignment = list(set(word_tokenize(Data_InstallationAlignment, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Installation/ Alignment ------------------
ENData_Security = (SafetyAudit.loc[SafetyAudit["Topic"] == "Security", "TransCleansingDetails"]).tolist() 
THData_Security = (SafetyAudit.loc[SafetyAudit["Topic"] == "Security", "CleansingDetails"]).tolist() 
Data_Security = ' '.join(ENData_Security+THData_Security) 

CountVector = CountVectorizer(stop_words = 'english', vocabulary = DictCorrect) 
doc_array = CountVector.fit_transform([Data_Security]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out())))   

ListWords_Security = list(set(word_tokenize(Data_Security, custom_dict=trie, engine='newmm')))

In [ ]:
# All_Encode_CountVector = model.encode(CountVector.get_feature_names_out()) 

# doc_array = doc_array[0]

# Temp_Encode_CountVector = []

# for i in range(len(All_Encode_CountVector)):
#     temp = []
#     temp.append(sum(All_Encode_CountVector[i]))
#     temp.append(doc_array[i])

#     Temp_Encode_CountVector.append(temp)
    
# Encode_CountVector = np.array(Temp_Encode_CountVector)

# print(All_Encode_CountVector)
# y = [1] * 1000
# y.extend([2] * 299)

# cmap_bold = ListedColormap(['#FF0000'])

# clf = NearestCentroid()
# clf.fit(Encode_CountVector, y)
# y_pred = clf.predict(Encode_CountVector)

# x_min, x_MostSimTOF = Encode_CountVector[:, 0].min() - 1, Encode_CountVector[:, 0].MostSimTOF() + 1
# y_min, y_MostSimTOF = Encode_CountVector[:, 1].min() - 1, Encode_CountVector[:, 1].MostSimTOF() + 1

# xx, yy = np.meshgrid(np.arange(x_min, x_MostSimTOF, 0.2), np.arange(y_min, y_MostSimTOF, 0.2))

# Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

# Z = Z.reshape(xx.shape)
# plt.figure()

# plt.scatter(Encode_CountVector[:, 0], Encode_CountVector[:, 1], c=doc_array, cmap=cmap_bold, edgecolor='k', s=20)
# plt.title("Cluster Unsafe Action")

# plt.show()

Show Cluster in Graph

In [ ]:
# from statistics import mean

# # Mean_Cluster = mean(list(doc_array))

# Cluster_UnsafeAction = [] 
# Cluster_UnsafeCondition = []
# Cluster_NearMiss = []
# Cluster_HNM = []
# Cluster_Accident = []

# for i in range(len(ListWords_UnsafeAction)):
#     temp = []
#     temp.append(ListWords_UnsafeAction[i])
#     temp.append(doc_array[i])
#     Cluster_UnsafeAction.append(temp)

# for i in range(len(ListWords_UnsafeCondition)):    
#     temp = []
#     temp.append(ListWords_UnsafeCondition[i])
#     temp.append(doc_array[i])
#     Cluster_UnsafeCondition.append(temp)

# for i in range(len(ListWords_NearMiss)):
#     temp.append(ListWords_NearMiss[i])
#     temp.append(doc_array[i])
#     Cluster_NearMiss.append(temp)

# for i in range(len(ListWords_HNM)):
#     temp.append(ListWords_HNM[i])
#     temp.append(doc_array[i])
#     Cluster_HNM.append(temp)

# for i in range(len(ListWords_Accident)):
#     temp.append(ListWords_Accident[i])
#     temp.append(doc_array[i])
#     Cluster_Accident.append(temp)    
    
# Cluster_UnsafeAction = pd.DataFrame(data = Cluster_UnsafeAction, columns = ['words', 'frequency'])
# Cluster_UnsafeCondition = pd.DataFrame(data = Cluster_UnsafeCondition, columns = ['words', 'frequency'])
# Cluster_NearMiss = pd.DataFrame(data = Cluster_NearMiss, columns = ['words', 'frequency'])
# Cluster_HNM = pd.DataFrame(data = Cluster_HNM, columns = ['words', 'frequency'])
# Cluster_Accident = pd.DataFrame(data = Cluster_Accident, columns = ['words', 'frequency'])

TF-IDF

In [ ]:
## Get TF-IDF Value of Every Cluster

# docs = [ListWords_UnsafeAction, ListWords_UnsafeCondition, ListWords_NearMiss, ListWords_HNM, ListWords_Accident]

# vocab = set(ListWords_UnsafeAction + ListWords_UnsafeCondition + ListWords_NearMiss + ListWords_HNM + ListWords_Accident)

# def tfidf(word, sentence):
#     tf = sentence.count(word) / len(sentence)
#     idf = np.log10(len(docs) / sum([1 for doc in docs if word in doc]))
#     return round(tf*idf, 4)

# TFIDF_UnsafeAction = []
# TFIDF_UnsafeCondition = []
# TFIDF_NearMiss = []
# TFIDF_Hnm = []
# TFIDF_Accident = []

# for word in vocab:
#     temp = []
#     temp.append(word)
#     temp.append(tfidf(word, ListWords_UnsafeAction))
#     TFIDF_UnsafeAction.append(temp)

#     temp = []
#     temp.append(word)
#     temp.append(tfidf(word, ListWords_UnsafeCondition))
#     TFIDF_UnsafeCondition.append(temp)

#     temp = []
#     temp.append(word)
#     temp.append(tfidf(word, ListWords_NearMiss))
#     TFIDF_NearMiss.append(temp)

#     temp = []
#     temp.append(word)
#     temp.append(tfidf(word, ListWords_HNM))
#     TFIDF_Hnm.append(temp)
    
#     temp = []
#     temp.append(word)
#     temp.append(tfidf(word, ListWords_Accident))
#     TFIDF_Accident.append(temp)

Find Frequency of each cluster of TypeOfFinding

In [ ]:
def CreateFrequency(word):
    Value = 6
    if word in ListWords_UnsafeAction:
        Value -= 1
    if word in ListWords_UnsafeCondition:
        Value -= 1
    if word in ListWords_NearMiss:
        Value -= 1
    if word in ListWords_HNM:
        Value -= 1
    if word in ListWords_Accident:
        Value -= 1
    if Value == 6:
        return 0
    else:
        return Value

Cluster_UnsafeAction = []
Cluster_UnsafeCondition = []
Cluster_NearMiss = []
Cluster_HNM = []
Cluster_Accident = []

for word in ListWords_UnsafeAction:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_UnsafeAction.append(temp)

for word in ListWords_UnsafeCondition:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_UnsafeCondition.append(temp)

for word in ListWords_NearMiss:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_NearMiss.append(temp)

for word in ListWords_HNM:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_HNM.append(temp)

for word in ListWords_Accident:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_Accident.append(temp)

Find Frequency of each cluster of topic

In [ ]:
def CreateFrequency(word):
    Value = 29
    if word in ListWords_LOTO:
        Value -= 1
    if word in ListWords_WAH:
        Value -= 1
    if word in ListWords_Scaffolding:
        Value -= 1
    if word in ListWords_Transportation:
        Value -= 1
    if word in ListWords_PTW_JSA:
        Value -= 1
    if word in ListWords_ProcessOperation:
        Value -= 1
    if word in ListWords_Radiation:
        Value -= 1
    if word in ListWords_Others:
        Value -= 1
    if word in ListWords_Lifting:
        Value -= 1
    if word in ListWords_Housekeeping:
        Value -= 1
    if word in ListWords_ToolsEquipment:
        Value -= 1
    if word in ListWords_HotWork:
        Value -= 1
    if word in ListWords_Excavation:
        Value -= 1
    if word in ListWords_CSE:
        Value -= 1
    if word in ListWords_ElectricalGrounding:
        Value -= 1
    if word in ListWords_PaintCoatBlast:
        Value -= 1
    if word in ListWords_ChemicalWork:
        Value -= 1
    if word in ListWords_SafetyManagement:
        Value -= 1
    if word in ListWords_PPE:
        Value -= 1
    if word in ListWords_WaterJet:
        Value -= 1
    if word in ListWords_PressureTest:
        Value -= 1
    if word in ListWords_SLPerformance:
        Value -= 1
    if word in ListWords_WorkProcedure:
        Value -= 1
    if word in ListWords_Civil:
        Value -= 1
    if word in ListWords_Insulation:
        Value -= 1
    if word in ListWords_Environmental:
        Value -= 1
    if word in ListWords_InstallationAlignment:
        Value -= 1
    if word in ListWords_Security:
        Value -= 1
    if Value == 28:
        return 0
    else:
        return Value

Cluster_LOTO = []
Cluster_WAH = []
Cluster_Scaffolding = []
Cluster_Transportaion = []
Cluster_PTWJSA = []
Cluster_ProcessOperation = []
Cluster_Radiation = []
Cluster_Others = []
Cluster_Lifting = []
Cluster_Housekeeping = []
Cluster_ToolsEquipment = []
Cluster_HotWork = []
Cluster_Excavation = []
Cluster_CSE = []
Cluster_ElectricalGrounding = []
Cluster_PaintCoatBlast = []
Cluster_ChemicalWork = []
Cluster_SafetyManagement = []
Cluster_PPE = []
Cluster_WaterJet = []
Cluster_PressureTest = []
Cluster_SLPerformance = []
Cluster_WorkProcedure = []
Cluster_Civil = []
Cluster_Insulation = []
Cluster_Environmental = []
Cluster_InstallationAlignment = []
Cluster_Security = []

for word in ListWords_LOTO:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_LOTO.append(temp)

for word in ListWords_WAH:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_WAH.append(temp)

for word in ListWords_Scaffolding:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_Scaffolding.append(temp)

for word in ListWords_Transportation:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_Transportaion.append(temp)

for word in ListWords_PTW_JSA:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_PTWJSA.append(temp)

for word in ListWords_ProcessOperation:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_ProcessOperation.append(temp)

for word in ListWords_Others:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_Others.append(temp)

for word in ListWords_Lifting:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_Lifting.append(temp)

for word in ListWords_Housekeeping:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_Housekeeping.append(temp)

for word in ListWords_ToolsEquipment:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_ToolsEquipment.append(temp)

for word in ListWords_HotWork:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_HotWork.append(temp)

for word in ListWords_Excavation:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_Excavation.append(temp)

for word in ListWords_CSE:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_CSE.append(temp)

for word in ListWords_ElectricalGrounding:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_ElectricalGrounding.append(temp)

for word in ListWords_PaintCoatBlast:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_PaintCoatBlast.append(temp)

for word in ListWords_ChemicalWork:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_ChemicalWork.append(temp)

for word in ListWords_SafetyManagement:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_SafetyManagement.append(temp)

for word in ListWords_PPE:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_PPE.append(temp)

for word in ListWords_WaterJet:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_WaterJet.append(temp)

for word in ListWords_PressureTest:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_PressureTest.append(temp)

for word in ListWords_SLPerformance:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_SLPerformance.append(temp)

for word in ListWords_WorkProcedure:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_WorkProcedure.append(temp)

for word in ListWords_Civil:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_Civil.append(temp)

for word in ListWords_Insulation:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_Insulation.append(temp)

for word in ListWords_Environmental:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_Environmental.append(temp)

for word in ListWords_InstallationAlignment:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_InstallationAlignment.append(temp)

for word in ListWords_Security:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_Security.append(temp)

1. เปลี่ยนรูปแบบการดึงข้อมูล tbFinding ให้ดึงจาก Database 
2. เพิ่มการแปลภาษา tbFinding ก่อนดึงข้อมูลมาจาก Database

In [ ]:
import pyodbc
import csv

TotalData = (pd.read_csv('./SMIT_Data/TotalData.csv', encoding='utf-8'))

TotalOldClassification_Finding = TotalData['Old'].tolist()[0]
TotalLatestClassification_Finding = TotalData['Latest'].tolist()[0]

ClassifyTopic = []

tbFindingNo = []
tbFindingArea = tbFinding['Area'].tolist()
tbFindingSubArea = []
tbFindingContractor = []
tbFindingTof = []
tbFindingTopic = []
tbFindingFinding = tbFinding['Finding'].tolist()

for index in range(len(Translate_tbFinding)):
    Data = word_tokenize(Translate_tbFinding[index], custom_dict=trie, engine='newmm')
    Finding = "-"
    Topic = "-"
    TempMostSimTOF = 0
    MostSimTOF = -1

    MostSimTopic = -1
    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_UnsafeAction])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_UnsafeAction]):
                tempMostSimTOF += Cluster_UnsafeAction[ListWords_UnsafeAction.index(i)][1]
        if TempMostSimTOF > MostSimTOF:
            MostSimTOF = tempMostSimTOF
            Finding = "Unsafe Action"
            TempMostSimTopic = 0

    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_UnsafeCondition])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_UnsafeCondition]):
                tempMostSimTOF += Cluster_UnsafeCondition[ListWords_UnsafeCondition.index(i)][1]
        if tempMostSimTOF > MostSimTOF:
            MostSimTOF = tempMostSimTOF
            Finding = "Unsafe Condition"
            tempMostSimTOF = 0
 
    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_NearMiss])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_NearMiss]):
                tempMostSimTOF += Cluster_NearMiss[ListWords_NearMiss.index(i)][1]
        if tempMostSimTOF > MostSimTOF:
            MostSimTOF = tempMostSimTOF
            Finding = "Near Miss"
            tempMostSimTOF = 0

    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_HNM])) > 3:
        for i in set([x.lower() for x in Data]):
            if i in [j[0] for j in Cluster_HNM]:
                tempMostSimTOF += Cluster_HNM[ListWords_HNM.index(i)][1]
        if tempMostSimTOF > MostSimTOF:
            MostSimTOF = tempMostSimTOF
            Finding = "HNM"
            tempMostSimTOF = 0

    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_Accident])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_Accident]):
                tempMostSimTOF += Cluster_Accident[ListWords_Accident.index(i)][1]
        if tempMostSimTOF > MostSimTOF:
            MostSimTOF = tempMostSimTOF
            Finding = "Accident"
            tempMostSimTOF = 0
      
    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_LOTO])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_LOTO]):
                TempMostSimTopic += Cluster_LOTO[ListWords_LOTO.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "LOTO/ LB"
            TempMostSimTopic = 0  
    
    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_WAH])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_WAH]):
                TempMostSimTopic += Cluster_WAH[ListWords_WAH.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "Work at height"
            TempMostSimTopic = 0

    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_Scaffolding])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_Scaffolding]):
                TempMostSimTopic += Cluster_Scaffolding[ListWords_Scaffolding.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "Scaffolding"
            TempMostSimTopic = 0

    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_Transportaion])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_Transportaion]):
                TempMostSimTopic += Cluster_Transportaion[ListWords_Transportation.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "Transportation"
            TempMostSimTopic = 0

    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_PTWJSA])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_PTWJSA]):
                TempMostSimTopic += Cluster_PTWJSA[ListWords_PTW_JSA.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "PTW & JSA"
            TempMostSimTopic = 0

    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_ProcessOperation])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_ProcessOperation]):
                TempMostSimTopic += Cluster_ProcessOperation[ListWords_ProcessOperation.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "Process & Operation"
            TempMostSimTopic = 0

    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_Radiation])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_Radiation]):
                TempMostSimTopic += Cluster_Radiation[ListWords_Radiation.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "Radiation"
            TempMostSimTopic = 0

    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_Others])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_Others]):
                TempMostSimTopic += Cluster_Others[ListWords_Others.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "Others"
            TempMostSimTopic = 0

    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_Lifting])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_Lifting]):
                TempMostSimTopic += Cluster_Lifting[ListWords_Lifting.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "Lifting"
            TempMostSimTopic = 0

    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_Housekeeping])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_Housekeeping]):
                TempMostSimTopic += Cluster_Housekeeping[ListWords_Housekeeping.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "Housekeeping"
            TempMostSimTopic = 0

    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_ToolsEquipment])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_ToolsEquipment]):
                TempMostSimTopic += Cluster_ToolsEquipment[ListWords_ToolsEquipment.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "Tools & Equipment"
            TempMostSimTopic = 0

    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_HotWork])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_HotWork]):
                TempMostSimTopic += Cluster_HotWork[ListWords_HotWork.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "Hot Work"
            TempMostSimTopic = 0

    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_Excavation])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_Excavation]):
                TempMostSimTopic += Cluster_Excavation[ListWords_Excavation.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "Excavation"
            TempMostSimTopic = 0

    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_CSE])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_CSE]):
                TempMostSimTopic += Cluster_CSE[ListWords_CSE.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "CSE"
            TempMostSimTopic = 0

    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_ElectricalGrounding])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_ElectricalGrounding]):
                TempMostSimTopic += Cluster_ElectricalGrounding[ListWords_ElectricalGrounding.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "Electrical & Grounding"
            TempMostSimTopic = 0

    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_PaintCoatBlast])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_PaintCoatBlast]):
                TempMostSimTopic += Cluster_PaintCoatBlast[ListWords_PaintCoatBlast.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "Paint/ Coat/ Blast"
            TempMostSimTopic = 0

    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_ChemicalWork])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_ChemicalWork]):
                TempMostSimTopic += Cluster_ChemicalWork[ListWords_ChemicalWork.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "Chemical Work"
            TempMostSimTopic = 0

    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_SafetyManagement])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_SafetyManagement]):
                TempMostSimTopic += Cluster_SafetyManagement[ListWords_SafetyManagement.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "Safety Management"
            TempMostSimTopic = 0

    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_PPE])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_PPE]):
                TempMostSimTopic += Cluster_PPE[ListWords_PPE.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "PPE"
            TempMostSimTopic = 0

    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_WaterJet])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_WaterJet]):
                TempMostSimTopic += Cluster_WaterJet[ListWords_WaterJet.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "Water jet"
            TempMostSimTopic = 0

    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_PressureTest])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_PressureTest]):
                TempMostSimTopic += Cluster_PressureTest[ListWords_PressureTest.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "Pressure test"
            TempMostSimTopic = 0

    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_SLPerformance])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_SLPerformance]):
                TempMostSimTopic += Cluster_SLPerformance[ListWords_SLPerformance.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "SL Performance"
            TempMostSimTopic = 0
            
    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_WorkProcedure])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_WorkProcedure]):
                TempMostSimTopic += Cluster_WorkProcedure[ListWords_WorkProcedure.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "Work Procedure"
            TempMostSimTopic = 0
            
    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_Civil])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_Civil]):
                TempMostSimTopic += Cluster_Civil[ListWords_Civil.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "Civil"
            TempMostSimTopic = 0
            
    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_Insulation])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_Insulation]):
                TempMostSimTopic += Cluster_Insulation[ListWords_Insulation.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "Insulation"
            TempMostSimTopic = 0
                        
    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_Environmental])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_Environmental]):
                TempMostSimTopic += Cluster_Environmental[ListWords_Environmental.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "Environmental"
            TempMostSimTopic = 0
                        
    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_InstallationAlignment])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_InstallationAlignment]):
                TempMostSimTopic += Cluster_InstallationAlignment[ListWords_InstallationAlignment.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "Installation/ Alignment"
            TempMostSimTopic = 0
                        
    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_Security])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_Security]):
                TempMostSimTopic += Cluster_Security[ListWords_Security.index(i)][1]
        if TempMostSimTopic > MostSimTopic:
            MostSimTopic = TempMostSimTopic
            Topic = "Security"
            TempMostSimTopic = 0

    tbFindingNo.append(str(TotalOldClassification_Finding+index+1)) 
    tbFindingSubArea.append('Unknown')  
    tbFindingContractor.append('Unknown')     
    tbFindingTof.append(Finding)
    tbFindingTopic.append(Topic)

connect_db = pyodbc.connect(Driver = "ODBC Driver 17 for SQL Server",
                            Server = "TONY",
                            Database = "SMIT3-Real",
                            uid = 'Local_SMIT3.0',
                            pwd = 'Tony123456',
                            Trusted_Connection = 'yes')      


Classification_TbFinding = list(zip(tbFindingNo, tbFindingArea, tbFindingSubArea, 
                                    tbFindingContractor, tbFindingTof, tbFindingTopic, 
                                    tbFindingFinding, Translate_tbFinding))

cursor = connect_db.cursor()
Query = "INSERT INTO [Classification_TbFinding] VALUES (?, ?, ?, ?, ?, ?, ?, ?);"

cursor.executemany(Query, Classification_TbFinding)

connect_db.commit()

NameError: name 'Translate_tbFinding' is not defined

In [ ]:
import csv

Old_Size = len(tbFindingFinding)

Head = ['Source', 'Old', 'Latest']

UpdateSize = [
              ['Classfication_TbFinding', TotalOldClassification_Finding, TotalLatestClassification_Finding+Old_Size]
             ] 

with open('./SMIT_Data/TotalData.csv', 'w', newline='', encoding="utf-8") as f:
  write = csv.writer(f)
  write.writerow(Head)
  write.writerows(UpdateSize)